In [7]:
import os
import glob
import json
import pandas as pd
from datetime import datetime
from row import Row

import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, trim
from pyspark.sql.types import IntegerType, LongType, FloatType
from pandas_schema import Column, Schema
import pandas_schema.validation as validation 

from cleaning import validation_schema

# Cleaning

This step will allow you to remove invalid data from the datasets. An example of invalid data is a negative value for a data representing a distance. For each column in the 4 sub-datasets, assertion are maded based on the TLC specifications.

### 1. Identification of the assertions

<img src="img/assertion_fhv_fhvhv.png" width="800" align="left"/>
<img src="img/assertion_green.png" width="800" align="left"/>
<img src="img/assertion_yellow.png" width="800" align="left"/>

### 2. Pandas-schema

A python module that checks such assertion exists: _panda_schema_. This module is very easy to use, an example is given for the FHV dataset (the implementation for the other sub-datasets can be found in the _cleaning.py_ file).

In [ ]:
from pandas_schema import Column, Schema
import pandas_schema.validation as validation 

# validation schema for fhv dataset
schema_fhv = Schema([
    Column('dispatching_base_num', 
           [validation.MatchesPatternValidation('^B[0-9]{5}$')], 
           allow_empty=True),
    Column('pickup_datetime', 
           [validation.DateFormatValidation('%Y-%m-%d %H:%M:%S')],
           allow_empty=True),
    Column('dropoff_datetime', 
           [validation.DateFormatValidation('%Y-%m-%d %H:%M:%S')],
           allow_empty=True),
    Column('pulocationid',
           [validation.InRangeValidation(1, 266)],
           allow_empty=True),
    Column('dolocationid', 
           [validation.InRangeValidation(1, 266)],
           allow_empty=True),
    Column('sr_flag', 
           [validation.InListValidation([1, None])],
           allow_empty=True)
])


def validate(df, validation_schema):

    """
    Validate the dataframe and return infos of the errors

    :param df: df to validate
    :param validation_schema: schema to validate the data 
    :return: list with errors info
    """

    # validate the data
    errors = validation_schema.validate(df)
    errors = [[error.row, error.column, error.value] for error in errors]

    return errors


The _validate_ function is a static method of the Row class

### 3. Implementation

#### 3.1. Basic cleaning operations

Some records contain errors that are not validity errors but display errors. These errors are :

* trailing space: "B0005 &nbsp; &nbsp; &nbsp;"
* leading space: "&nbsp; &nbsp; &nbsp; B0005"
* int/float represented as a string: "12"
* replace empty cell of numeric column by: 0

This is done by loading each .csv file in a spark dataframe. We apply the following operations on the dataframe:

* trim: to remove leading and trailing whithespace.
* cast: to convert string to float/int
* fillna: to fill the empty cells of some numeric columns by 0.

A dictionary is used to specify on which columns to apply these last two operations.

#### 3.2. Validation step

A version of pandas-schema that runs on the pyspark dataframe does not seem to exist at the moment. So we need to load the pyspark dataframe on the main node using the _toPandas()_ method. For large files the dataframe is too large for the main node's RAM. We didn't solve this problem but we considered two solutions:

* increase the memory of the main node
* split the file into several parts and reassemble it at the end

When the dataframe is loaded on the main node, the validation scheme corresponding to the appropriate dataset is then applied. 

With the errors generated by this validation step, 2 things can be done:

* Rows with errors are deleted.
* Errors are saved to generate statistics on them.

In [8]:
os.environ['HADOOP_CONF_DIR']="/etc/hadoop/conf"

# python configuration
os.environ['PYSPARK_PYTHON']="/usr/local/anaconda3/bin/python"
os.environ['PYSPARK_DRIVER_PYTHON']="/usr/local/anaconda3/bin/python"

from pyspark.sql import SparkSession
from pyspark import SparkFiles, SQLContext


# remove old spark session
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass

# Create a new spark session, with YARN as resource manager, requesting 4 worker nodes.
spark = SparkSession \
    .builder \
    .master("yarn") \
    .config("spark.executor.instances","4") \
    .appName("project_ceci18") \
    .getOrCreate()

# Create a sql spark context
sc=spark.sparkContext
sqlContext = SQLContext(sc)

Spark application already started. Terminating existing application and starting new one


In [9]:
# define the cleaning configurations

cleaning_conf = {
    'fhv':{
        'cast': {
            'int': ['pulocationid', 'dolocationid', 'sr_flag']
        },
        'fill': {
        }
    },
    'fhvhv':{
        'cast': {
            'int': ['pulocationid', 'dolocationid', 'sr_flag']
        },
        'fill': {
        }
    },
    'green': {
        'cast': {
            'int': ['pulocationid', 'dolocationid', 'vendorid', 'ratecodeid', 'passenger_count',
                    'payment_type', 'trip_type'],
            'float': ['trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 
                      'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount',
                      'congestion_surcharge']
        },
        'fill': {
            'extra':0, 'mta_tax':0, 'fare_amount':0, 
            'ehail_fee':0, 'tolls_amount':0
        }
    },
    'yellow': {
        'cast': {
            'int': ['pulocationid', 'dolocationid', 'vendorid', 'ratecodeid', 'passenger_count',
                    'payment_type'],
            'float': ['trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 
                      'tolls_amount', 'improvement_surcharge', 'total_amount',
                      'congestion_surcharge']
        },
        'fill': {
            'extra':0, 'mta_tax':0, 'fare_amount':0, 'tolls_amount':0
        }
    }
}


In [14]:
dataset = "yellow"

# get all the filename
hdfs_path = 'hdfs://public00:8020/user/hpda000034/infoh600/clean'
local_path = '/home/hpda00034/infoh600/sampled'
filenames = sorted(glob.glob("{}/{}_*.csv".format(local_path, dataset)))
filenames = [os.path.basename(filename) for filename in filenames]

# get the right cleaning configuration
cast = cleaning_conf[dataset]['cast'] 
fill = cleaning_conf[dataset]['fill'] 

# list to store all the errors
errors_df = []

filenames = ['yellow_tripdata_2015-01.csv']
for filename in filenames:
    print(filename)
    
    # load the dataframe and fill empty cell
    trips = sqlContext.read.csv("./integrated/{}/{}".format(dataset, filename), 
                                header=True,
                                inferSchema=True)\
                      .fillna(fill)
    
    # remove leading and traling whitespace
    for column in trips.columns:
        trips = trips.withColumn(column, trim(trips[column]))
    
    # cast numeric column
    for type_, columns in cast.items():
        for column in columns:
            if type_ == 'int':
                trips = trips.withColumn(column, trips[column].cast(IntegerType()))
            elif type_ == 'float':
                trips = trips.withColumn(column, trips[column].cast(FloatType()))

    # load in the main node
    df = trips.toPandas()
    df.head()

    # get errors for the current file
    errors = [error + [filename] for error 
               in Row.validate(df, validation_schema[dataset])]
    errors_df += errors
    
    # get dirty row
    rows = [error[0] for error in errors]
    
    # remove dirty rows
    if len(rows) != 0:
        df = df.drop(rows, axis=0)
        df.to_csv('../clean/{}/{}'.format('buffer', filename), index=False, header=True)
    else:
        df.to_csv('../clean/{}/{}'.format(dataset, filename), index=False, header=True)

# save the errors
columns = ['row', 'column', 'value', 'file']
errors_df = pd.DataFrame(errors_df, columns=columns)
errors_df.to_csv('../invalid_data/{}.csv'.format(dataset), index=False)

yellow_tripdata_2015-01.csv


In [15]:
# Stop spark
try: 
    spark.stop()
except: 
    pass

We then send the cleaned-up files to HDFS.

A problem was encountered during this step. When files with rows that have been deleted are saved in the same folder as files without deleted rows, transferring files with deleted rows to HDFS does not work (checksum problem). Therefore, these two types of files have been saved in different folders. This problem deserves further investigation, but we didn't have the time and we'll settle for this do-it-yourself solution.

In [ ]:
!hadoop fs -copyFromLocal /home/ceci18/clean/yellow/* ./clean/yellow
!hadoop fs -copyFromLocal /home/ceci18/clean/buffer/* ./clean/yellow
!rm /home/ceci18/clean/buffer/*

### 4. Statistics

In this section we will summarize the validation errors found. For each dataset we will represent the count of errors according to the different files and the count of the errors according to the different columns.

NB: to see the graphs in better quality, you have to go to the figures folder.

NB2: please note that the graphs do not show the number of rows deleted but the number of errors (there may be several errors per row).

In [ ]:
dataset = "green"

path = '/home/ceci18/invalid_data'

df = pd.read_csv('{}/{}.csv'.format(path, dataset))

if df.shape[0] > 0:
    fig, (ax1, ax2) = plt.subplots(2,1, figsize = (26,22)) # yellow/green (26,22), fhv (20,13)
    fig.tight_layout(pad=28.0)

    errors_by_column = df.groupby(['column']).size()
    errors_by_column.plot.bar(ax=ax1)
    ax1.set_title('Count of errors by column', fontsize=18)
    ax1.set_xlabel('column', fontsize=18)
    ax1.set_ylabel('count', fontsize=18)
    plt.setp(ax1.get_xticklabels(), rotation='vertical', fontsize=18)
    plt.setp(ax1.get_yticklabels(), fontsize=18)
    
    errors_by_file = df.groupby(['file']).size()
    errors_by_file.plot.bar(ax=ax2)
    ax2.set_title('Count of errors by file', fontsize=18)
    ax2.set_xlabel('file', fontsize=18)
    ax2.set_ylabel('count', fontsize=18)
    plt.setp(ax2.get_xticklabels(), rotation='vertical', fontsize=18)
    plt.setp(ax2.get_yticklabels(), fontsize=18)
    
    fig.suptitle('{} dataset'.format(dataset.capitalize()), fontsize=26)
    
    plt.savefig('figures/errors_{}.png'.format(dataset))
    plt.close()
    
else:
    print("no error for {}".format(dataset))

#### 4.1. FHV

<img src="figures/errors_fhv.png" width="800" align="left"/>

The vast majority of errors come from the **sr_flag** column.  Based on the TLC specifications, this entry can take the value 1 or be empty. Some rows had other values : ['4.0', '3.0', '2.0', '5.0', '6.0', '7.0', '13.0', '8.0', '10.0','9.0', '12.0', '16.0', '17.0', '11.0', '32.0', '15.0', '14.0','21.0', '20.0', '24.0', '22.0', '19.0', '18.0', '29.0','36.0', '41.0', '28.0']. All these mistakes were made in the same month : 01/2019. 

The other error found concerns the **dispatching_base_num** column. Some rows have the value '\N'.

These two types of errors cannot be repaired. The associated rows are therefore removed. A total of 150356 rows were removed in the _fhv_tripdata_2019-01.csv_ file and 5 in the _fhv_tripdata_2019-02.csv_ file.

#### 4.2. Green

<img src="figures/errors_green.png" width="800" align="left"/>

The errors for columns **congestion_surcharge**, **ehail_fee**, **extra**, **fare_amount**, **improvement_surcharge**, **mta_tax**, **passenger_count**, **tip_amount**, **tolls_amount**, **total_amount** and **trip_distance** are all of the same type: the entries must be positive but for some rows they are negative.

For the **ratecodid** column, TLC specifies that the input must take an integer value from 1 to 6. But some entries have the value 99.

A total of 6,667 rows were deleted on all the _green_ files.

#### 4.3. Yellow

<img src="figures/errors_yellow.png" width="800" align="left"/>

The errors for columns **congestion_surcharge**, **extra**, **fare_amount**, **improvement_surcharge**, **tip_amount**, **tolls_amount** and **total_amount** are all of the same type: the entries must be positive but for some rows they are negative.

The **payment_type** column must take integer values from 1 to 6 but for some rows it has one of the following values : [192., 207., 161., 138., 98., 263., 131., 228., 237., 124., 69., 231., 149., 238., 48., 157., 74., 33., 190., 146., 50., 229., 245., 115., 222., 143., 45., 162., 230., 139., 165., 90., 252., 113., 152., 24., 76., 260., 159., 142., 61., 234., 82., 169., 108., 227., 163., 262., 186., 101., 235., 144., 79., 213., 164., 226., 35., 258., 133., 223., 141., 114., 145., 14., 181., 128., 36., 148., 88., 211., 103., 129., 233., 240., 206., 72., 232.,137., 170., 167., 120., 16., 249., 7., 91., 25., 175., 117., 209., 31., 86.]

For the **ratecodid** column, TLC specifies that the input must take an integer value from 1 to 6. But some entries have the value 99.

For the **vendorid** column, the entries must be equal to 1 or 2 but some entries have one of this values: [3., 4.]

A total of 53127 rows were deleted on all the yellow files.